In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
from sqlalchemy import create_engine
import re
import requests as req
import json
import datetime
import time

In [2]:
KEY = 'b50bc5-5af08a'

BCN = 'BCN'

status = 'landed'

url = f"http://aviation-edge.com/v2/public/timetable?key={KEY}&iataCode={BCN}&type=departure&status={status}"


## Get flight information request

In [3]:
def get_data(airport,status = 'landed'):
    
    KEY = 'b50bc5-5af08a'
    url = f"http://aviation-edge.com/v2/public/timetable?key={KEY}&iataCode={airport}&type=departure&status={status}"    
    response = req.get(url)
    print(response)
    print((datetime.datetime.now()))
    json_file = response.json()

    if type(json_file) == list:
        df_f = pd.DataFrame(json_file)

        return df_f
    
    

In [4]:
def clean_data(data_df):
    data_df["airln_name"] = data_df["airline"].apply(lambda x: x['name'])
    data_df["airln_code"] = data_df["airline"].apply(lambda x: x['iataCode'])
    data_df['arri_delay'] = data_df['arrival'].apply(lambda x: x['delay'])
    data_df['arri_airpt'] = data_df['arrival'].apply(lambda x: x['iataCode'])
    data_df['schd_arri_time'] = data_df['arrival'].apply(lambda x: x['scheduledTime'])
    data_df['codeshared'] = data_df['codeshared'].apply(lambda x: x['airline']['name'] if x else None)
    data_df['depart_delay'] = data_df['departure'].apply(lambda x: x['delay'])
    data_df['depart_airpt'] = data_df['departure'].apply(lambda x: x['iataCode'])
    data_df['schd_depart_time'] = data_df['departure'].apply(lambda x: x['scheduledTime'])
    data_df['depart_gate'] = data_df['departure'].apply(lambda x: x['gate'])
    data_df['depart_terminal'] = data_df['departure'].apply(lambda x: x['terminal'])
    data_df['icaonumb'] = data_df['flight'].apply(lambda x: x['icaoNumber'])
    data_df['flight'] = data_df['flight'].apply(lambda x: x['number'])
    data_df = data_df.drop(['airline','arrival','departure'], axis =1)
    
    return data_df
    

## Get airports by country

In [5]:
def get_data_airpts(country_list_code):
    
    
    for code in country_list_code:
    
        KEY = 'b50bc5-5af08a'
        url_airpt = f'https://aviation-edge.com/v2/public/airportDatabase?key={KEY}&codeIso2Country={code}'

        response = req.get(url_airpt)
        print(response)
        print((datetime.datetime.now()))
        json_file = response.json()
        df = pd.DataFrame(json_file)
        return df

In [6]:
def get_data_airpts(country):
    
    KEY = 'b50bc5-5af08a'
    url_airpt = f'https://aviation-edge.com/v2/public/airportDatabase?key={KEY}&codeIso2Country={country}'
    
    response = req.get(url_airpt)
    print(response)
    print((datetime.datetime.now()))
    json_file = response.json()
    df = pd.DataFrame(json_file)
    return df

In [7]:
spain_airpts = get_data_airpts('ES')

<Response [200]>
2019-07-29 09:15:02.353142


In [8]:
spain_airpts.head(2)

,GMT,airportId,codeIataAirport,codeIataCity,codeIcaoAirport,codeIso2Country,geonameId,latitudeAirport,longitudeAirport,nameAirport,nameCountry,phone,timezone
0,1,27,ABC,ABC,LEAB,ES,6299323,38.948334,-1.863333,Los Llanos,Spain,,Europe/Madrid
1,0,55,ACE,ACE,GCRR,ES,6297189,28.950668,-13.609059,Lanzarote,Spain,902 404 704,Atlantic/Canary


In [ ]:
"""def get_iata_codes(airpts_lst):
    
    for i in airpts_lst:
        
    airpts_lst = df_airpts['codeIataAirport'].to_list()
    country_code = df_airpts['codeIso2Country'].to_list()
    name = df_airpts['nameAirport'].to_list()
    return airpts_lst,country_code, name
    
 """   

In [9]:
def get_iata_codes(df_airpts):
    airpts_lst = df_airpts['codeIataAirport'].to_list()
    country_code = df_airpts['codeIso2Country'].to_list()
    name = df_airpts['nameAirport'].to_list()
    return airpts_lst,country_code, name

### Spain

In [10]:
spain_airpts_lst, country_code_sp, name_airport = get_iata_codes(df_airpts=spain_airpts)


In [11]:
print(spain_airpts_lst)

['ABC', 'ACE', 'AEI', 'AGP', 'ALC', 'BCN', 'BIO', 'BJZ', 'CDT', 'CDZ', 'EAS', 'EEL', 'EEM', 'EER', 'EEU', 'FES', 'FGR', 'FUE', 'GEN', 'GGA', 'GMZ', 'GRO', 'GRX', 'HEV', 'HSK', 'IBZ', 'ILD', 'JCU', 'LCG', 'LEI', 'LEN', 'LEU', 'LPA', 'MAD', 'MAH', 'MJV', 'MLN', 'NDO', 'ODB', 'OVD', 'OZI', 'OZL', 'OZP', 'OZU', 'PMI', 'PNA', 'PXS', 'QGN', 'QLQ', 'REU', 'RGS', 'RJL', 'RMU', 'ROZ', 'RRA', 'SCQ', 'SDR', 'SLM', 'SPC', 'SPO', 'SVQ', 'TFN', 'TFS', 'TOJ', 'UER', 'UTL', 'VDE', 'VGO', 'VIT', 'VLC', 'VLL', 'XIV', 'XJB', 'XJI', 'XJJ', 'XJN', 'XJO', 'XJR', 'XJU', 'XOC', 'XOJ', 'XOU', 'XQA', 'XRY', 'XTI', 'XTJ', 'XUA', 'XUF', 'XUT', 'XZZ', 'YJB', 'YJC', 'YJD', 'YJE', 'YJG', 'YJH', 'YJI', 'YJL', 'YJM', 'YJR', 'YJV', 'YJW', 'ZAZ', 'ZRC']


In [13]:
spain_airports = pd.DataFrame({'country_code':country_code_sp,
             'iata_code':spain_airpts_lst,
                              'name': name_airport})

In [13]:
#spain_airports.to_sql('airports',engine, if_exists='append', index=False)

### France

In [12]:
france_airpts = get_data_airpts('FR')

<Response [200]>
2019-07-29 09:15:24.899679


In [13]:
france_airpts_lst,country_code_fr, name_airport = get_iata_codes(df_airpts=france_airpts)

In [16]:
france_airports = pd.DataFrame({'country_code':country_code_fr,
             'iata_code':france_airpts_lst,
                              'name': name_airport})

In [17]:
#france_airports.to_sql('airports',engine, if_exists='append', index=False)

### Germany (DE)

In [14]:
germany_airpts = get_data_airpts('DE')

<Response [200]>
2019-07-29 09:15:33.430408


In [15]:
germany_airpts_lst,country_code_de, name_airport = get_iata_codes(df_airpts=germany_airpts)

In [19]:
germany_airports = pd.DataFrame({'country_code':country_code_de,
             'iata_code':germany_airpts_lst,
                              'name': name_airport})

In [21]:
#germany_airports.to_sql('airports',engine, if_exists='append', index=False)

### Italy

In [16]:
italy_airpts = get_data_airpts('IT')

<Response [200]>
2019-07-29 09:15:37.816071


In [17]:
italy_airpts_lst, country_code_it, name_airport = get_iata_codes(df_airpts=italy_airpts)

In [22]:
italy_airports = pd.DataFrame({'country_code':country_code_it,
             'iata_code':italy_airpts_lst,
                              'name': name_airport})

In [25]:
#italy_airports.to_sql('airports',engine, if_exists='append', index=False)

## All countries airports iata codes

In [18]:
countries_airpts_lst = spain_airpts_lst + france_airpts_lst + germany_airpts_lst + italy_airpts_lst

In [19]:
df_countries_airpt = pd.DataFrame(countries_airpts_lst)

In [20]:
df_countries_airpt.head()

,0
0,ABC
1,ACE
2,AEI
3,AGP
4,ALC


## Wednesday, July 24

In [32]:
dict_airpt_df = {}

for aipt_code in countries_airpts_lst:
    
    df_airpt = get_data(airport = f"{aipt_code}")
    
    if type(df_airpt) == pd.core.frame.DataFrame:
        df_airpt_clean = clean_data(df_airpt)
        dict_airpt_df[aipt_code] = df_airpt_clean
        time.sleep(2)


<Response [200]>
2019-07-29 16:58:27.100756
<Response [200]>
2019-07-29 16:58:27.793976
<Response [200]>
2019-07-29 16:58:30.262494
<Response [200]>
2019-07-29 16:58:30.915764
<Response [200]>
2019-07-29 16:58:33.702296
<Response [200]>
2019-07-29 16:58:36.568689
<Response [200]>
2019-07-29 16:58:39.352333
<Response [200]>
2019-07-29 16:58:42.003501
<Response [200]>
2019-07-29 16:58:44.550004
<Response [200]>
2019-07-29 16:58:45.041877
<Response [200]>
2019-07-29 16:58:45.700591
<Response [200]>
2019-07-29 16:58:48.172781
<Response [200]>
2019-07-29 16:58:48.673522
<Response [200]>
2019-07-29 16:58:49.129463
<Response [200]>
2019-07-29 16:58:49.611784
<Response [200]>
2019-07-29 16:58:50.053392
<Response [200]>
2019-07-29 16:58:50.808876
<Response [200]>
2019-07-29 16:58:51.473932
<Response [200]>
2019-07-29 16:58:54.009233
<Response [200]>
2019-07-29 16:58:54.516360
<Response [200]>
2019-07-29 16:58:55.136983
<Response [200]>
2019-07-29 16:58:57.748750
<Response [200]>
2019-07-29 16:5

<Response [200]>
2019-07-29 17:02:28.006147
<Response [200]>
2019-07-29 17:02:28.985480
<Response [200]>
2019-07-29 17:02:31.542333
<Response [200]>
2019-07-29 17:02:32.054411
<Response [200]>
2019-07-29 17:02:32.650525
<Response [200]>
2019-07-29 17:02:35.294120
<Response [200]>
2019-07-29 17:02:35.865113
<Response [200]>
2019-07-29 17:02:36.638380
<Response [200]>
2019-07-29 17:02:37.449476
<Response [200]>
2019-07-29 17:02:40.267044
<Response [200]>
2019-07-29 17:02:43.355082
<Response [200]>
2019-07-29 17:02:44.155068
<Response [200]>
2019-07-29 17:02:44.872454
<Response [200]>
2019-07-29 17:02:45.851757
<Response [200]>
2019-07-29 17:02:46.602076
<Response [200]>
2019-07-29 17:02:49.289696
<Response [200]>
2019-07-29 17:02:49.812389
<Response [200]>
2019-07-29 17:02:50.311176
<Response [200]>
2019-07-29 17:02:51.480045
<Response [200]>
2019-07-29 17:02:52.278113
<Response [200]>
2019-07-29 17:02:52.978772
<Response [200]>
2019-07-29 17:02:53.733227
<Response [200]>
2019-07-29 17:0

<Response [200]>
2019-07-29 17:05:52.281181
<Response [200]>
2019-07-29 17:05:53.158578
<Response [200]>
2019-07-29 17:05:53.987295
<Response [200]>
2019-07-29 17:05:54.895546
<Response [200]>
2019-07-29 17:05:55.662749
<Response [200]>
2019-07-29 17:05:56.585248
<Response [200]>
2019-07-29 17:05:57.406596
<Response [200]>
2019-07-29 17:05:58.274871
<Response [200]>
2019-07-29 17:05:59.081114
<Response [200]>
2019-07-29 17:05:59.996121
<Response [200]>
2019-07-29 17:06:00.883870
<Response [200]>
2019-07-29 17:06:01.714403
<Response [200]>
2019-07-29 17:06:02.294679
<Response [200]>
2019-07-29 17:06:03.199473
<Response [200]>
2019-07-29 17:06:04.019076
<Response [200]>
2019-07-29 17:06:04.711239
<Response [200]>
2019-07-29 17:06:05.499094
<Response [200]>
2019-07-29 17:06:06.307093
<Response [200]>
2019-07-29 17:06:07.120119
<Response [200]>
2019-07-29 17:06:07.718287
<Response [200]>
2019-07-29 17:06:08.624344
<Response [200]>
2019-07-29 17:06:09.452786
<Response [200]>
2019-07-29 17:0

KeyboardInterrupt: 

In [33]:
from sqlalchemy import create_engine


driver = 'mysql+pymysql:'
user = 'root'
password = '19693903'
ip = '127.0.0.1'
database = 'project_4'

In [34]:
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)


In [35]:
def save_to_sql_clean():
    for key, value in dict_airpt_df.items():
        
        value = value.astype({'schd_arri_time': 'datetime64[ns]','schd_depart_time':'datetime64[ns]'})
        value = value[['airln_name', 'airln_code','type', 'codeshared', 'status', 'icaonumb', 'flight','depart_terminal',\
      'depart_gate', 'depart_airpt' ,'schd_depart_time','depart_delay','arri_airpt','schd_arri_time',\
      'arri_delay']]
        value['depart_delay'] = value['depart_delay'].fillna(0)
        value['arri_delay'] = value['arri_delay'].fillna(0)
        value = value.astype({'depart_delay': int,'arri_delay':int})
        value = value.drop_duplicates(subset = ['flight','depart_terminal','schd_depart_time', 'depart_delay', \
                              'schd_arri_time', 'arri_delay'], keep='first')
        
        value.to_sql('flights',engine, if_exists='append', index=False)

In [36]:
save_to_sql_clean()